In [ ]:
import re
import emoji
import unidecode
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

In [ ]:
df = pd.read_csv('')

In [ ]:

# --- Etapa 1: Limpeza do texto ---
def limpar_texto(texto):
    texto = texto.lower()
    texto = re.sub(r"http\S+", "", texto)
    texto = emoji.replace_emoji(texto, '')
    texto = re.sub(r"[^a-zA-Z0-9\s]", "", texto)
    texto = unidecode.unidecode(texto)
    texto = re.sub(r"\s+", " ", texto).strip()
    return texto

df['texto_limpo'] = df['texto_original'].apply(limpar_texto)

# --- Etapa 2: Embeddings com BERT ---
modelo_bert = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
df['vector'] = list(modelo_bert.encode(df['texto_limpo'].tolist(), show_progress_bar=True))

# --- Etapa 3: Comparar e consolidar similaridade ---
limiar_remocao = 0.85
indices_para_remover = set()
pares_similares = []

# mapeamento de index -> indices duplicados para somar frequência
agrupamento_similares = {}

print("\nComparando mensagens por similaridade...")
for i in tqdm(range(len(df))):
    if i in indices_para_remover:
        continue
    vec_i = df.iloc[i]['vector']
    for j in range(i + 1, len(df)):
        if j in indices_para_remover:
            continue
        vec_j = df.iloc[j]['vector']
        sim = cosine_similarity([vec_i], [vec_j])[0][0]
        if sim >= limiar_remocao:
            pares_similares.append((i, j, sim))
            indices_para_remover.add(j)
            if i not in agrupamento_similares:
                agrupamento_similares[i] = []
            agrupamento_similares[i].append(j)

# --- Etapa 4: Somar frequências das mensagens similares ---
for idx_base, indices_duplicados in agrupamento_similares.items():
    freq_original = df.at[idx_base, 'frequencia']
    soma = sum(df.loc[dup_idx, 'frequencia'] for dup_idx in indices_duplicados)
    df.at[idx_base, 'frequencia'] = freq_original + soma

# --- Etapa 5: Remover duplicadas ---
df = df.drop(index=list(indices_para_remover)).reset_index(drop=True)

# --- Etapa 6: Visualizar resultado final ---
print(f"\nMensagens agrupadas por similaridade (≥ {limiar_remocao}):\n")
for i, j, sim in sorted(pares_similares, key=lambda x: -x[2]):
    print(f"Similaridade: {sim:.4f}")
    print(f"A (mantida): {df.iloc[i]['texto_limpo']}")
    print(f"B (removida): {df.iloc[j]['texto_limpo']}\n")

print("\n### DataFrame final:")
print(df[['texto_original', 'frequencia']])
